In [6]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.experimental import BabyAGI


In [3]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null


Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


In [ ]:
curl https://hci-gpt-assistant.openai.azure.com/openai/deployments/hci-embedings/embeddings?api-version=2022-12-01\
  -H "Content-Type: application/json" \
  -H "api-key: 0c051dbede6f4f7cbe6e2de59214a1dc" \
  -d "{\"input\": \"The food was delicious and the waiter...\"}"

In [ ]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMem
oryDocstore


In [8]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})


In [20]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, LLMChain

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a detailed plan for the current objective. Write a plan how to solve this objective: {objective}"
)

code_prompt = PromptTemplate.from_template(
    "You are a senior software engineer who is an expert at writing code. Write code to solve this problem: {problem}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
code_chain = LLMChain(llm=OpenAI(temperature=0), prompt=code_prompt)
tools = [
    Tool(
        name = "Code",
        func = code_chain.run,
        description = "useful for when you need to write code. Input: a problem to solve. Output: code that solves the problem. Please be very clear what the problem is!",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with plan for objective task. Input: an objective to create a plan for. Output: a plan to accomplish the objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)


In [21]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)


In [23]:
OBJECTIVE = "Create a python flask server with example of Rest API. Also write a unit tests for the created flask server."


In [24]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 5
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, task_execution_chain=agent_executor, verbose=verbose, max_iterations=max_iterations
)


In [25]:
baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: What tasks need to be completed?
Action: TODO
Action Input: Create a python flask server with example of Rest API. Also write a unit tests for the created flask server.
Observation: 

Plan to Create a Python Flask Server with Example of Rest API and Unit Tests:

1. Research:
    a. Research the basics of Python Flask and Rest API.
    b. Research the best practices for creating a Python Flask server and Rest API.
    c. Research the best practices for writing unit tests for a Python Flask server and Rest API.

2. Setup:
    a. Install the necessary software and libraries for Python Flask and Rest API.
    b. Create a project directory and set up the necessary files and folders.

3. Create the Flask Server:
    a. Create the main Flask server file.
    b. Create the necessary routes and functions for the server.

4. Create the Rest API:
    a. Create the n

{'objective': 'Create a python flask server with example of Rest API. Also write a unit tests for the created flask server.'}